In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
import argparse
import logging
import os

# from ginkgo import invMass_ginkgo
# from invMass_ginkgo import *
from invMass_ginkgo_node import *
from ginkgo.utils import get_logger


logger = get_logger(level = logging.WARNING)
fh = logging.FileHandler('spam.log')
fh.setLevel(logging.WARNING)
logger.addHandler(fh)


"""
create node class
understand original output
simplify recursion logic further

define typology of a tree
access data on each node

call algo to generate tree
have a fully defined tree with 4D vectors for each node

generate likelihood for each tree 
compute eq (12)
eq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r

"""

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'\ncreate node class\nunderstand original output\nsimplify recursion logic further\n\ndefine typology of a tree\naccess data on each node\n\ncall algo to generate tree\nhave a fully defined tree with 4D vectors for each node\n\ngenerate likelihood for each tree \ncompute eq (12)\neq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r\n\n'

In [3]:
"""Parameters"""


rate2 = torch.tensor(8.)

# Parameters to get ~<10 constituents to test the trellis algorithm
pt_min = torch.tensor(4.**2)

### Physics inspired parameters to get ~ between 20 and 50 constituents
W_rate = 3.
QCD_rate = 1.5

QCD_mass = 30.
class ginkgo_simulator():
    def __init__(self,
                 rate,
                 pt_cut,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate= 1.5,
                ):
        
        self.root_rate = root_rate
        self.rate = rate
        self.pt_cut = pt_cut
        self.M2start = torch.tensor(M2start) # mass squared to start with
        self.Nsamples = Nsamples
        self.minLeaves = minLeaves
        self.maxLeaves = maxLeaves
        self.maxNTry = maxNTry
        self.jetType = jetType # W or QCD 
        self.jetM = np.sqrt(M2start) # mass to start with
        self.jetdir = np.array([1,1,1])
        self.jetP = jetP
        self.jetvec = self.jetP * self.jetdir / np.linalg.norm(self.jetdir)
        self.jet4vec = np.concatenate(([np.sqrt(self.jetP ** 2 + self.jetM ** 2)], self.jetvec))
        logger.debug(f"jet4vec = {self.jet4vec}")
        
        if jetType == "W":
            # defined in paper, W jets have a different root rate
            self.rate=torch.tensor([self.root_rate,self.rate])
        elif jetType == "QCD":
            # QCD jets maintain the same rate throughout
            self.rate=torch.tensor([self.rate,self.rate])
        else:
            raise ValueError("Choose a valid jet type between W or QCD")



    def simulator(self):

        simulator = Simulator(jet_p = self.jet4vec,
                                         pt_cut = float(self.pt_cut),
                                         Delta_0 = self.M2start,
                                         M_hard = self.jetM ,
                                         num_samples = int(self.Nsamples),
                                         minLeaves = int(self.minLeaves),
                                         maxLeaves = int(self.maxLeaves),
                                         maxNTry = int(self.maxNTry)
                                         )
        return simulator
       
    def generate(self):
        
        simulator = self.simulator()
        jet_list = simulator(self.rate)

        logger.debug(f"---"*10)
        logger.debug(f"jet_list = {jet_list}")
        
        return jet_list

In [4]:
# the range of leaves that you would consider as valid generations
minLeaves = 3
maxLeaves = 100

# number of jets you wish to generate
Nsamples = 1

# exponential rate parameter
rate = 1.5

# mass squared cut off to yield leaves
pt_cut =  torch.tensor(1.1**2)

# mass squared to start with
M2start = 80.**2

# the maximum times you are willing to try to get Nsamples
maxNTry = 1

jetP=400.

In [5]:
jetType ="QCD"


ginkgo = ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP)

QCD_jets = ginkgo.generate()

Node 0
 Vec4: [407.92156109 230.94010768 230.94010768 230.94010768]
 Decay Rate: 1
 Mass Squared: tensor(6400.)

Node 1
 Vec4: [162.55707764  82.8948317   98.39096924  97.8846915 ]
 Decay Rate: tensor(0.4617)
 Mass Squared: tensor(291.0548)

Node 2
 Vec4: [245.36448345 148.04527597 132.54913843 133.05541617]
 Decay Rate: tensor(0.4708)
 Mass Squared: tensor(3013.3081)

Node 3
 Vec4: [134.78091259  70.27104967  82.06698332  79.86704619]
 Decay Rate: tensor(0.3922)
 Mass Squared: tensor(114.1392)

Node 4
 Vec4: [27.77616051 12.62377972 16.32398318 18.01764257]
 Decay Rate: tensor(0.5176)
 Mass Squared: tensor(21.0474)

Node 5
 Vec4: [59.9122668  38.66396952 34.61985502 29.60490641]
 Decay Rate: tensor(0.2603)
 Mass Squared: tensor(19.5922)

Node 6
 Vec4: [185.45220385 109.38129874  97.92927651 103.45050283]
 Decay Rate: tensor(0.7089)
 Mass Squared: tensor(2136.1018)

Node 7
 Vec4: [115.56480801  60.70891929  70.30936579  68.06132908]
 Decay Rate: tensor(0.8227)
 Mass Squared: tensor(93.

In [6]:
root_rate = 4.
jetType ="W"


ginkgo= ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate)

W_jets = ginkgo.generate()

Node 0
 Vec4: [407.92156109 230.94010768 230.94010768 230.94010768]
 Decay Rate: 1
 Mass Squared: tensor(6400.)

Node 1
 Vec4: [251.80027154 151.17948979 119.17192671 162.12486536]
 Decay Rate: tensor(0.0228)
 Mass Squared: tensor(61.7185)

Node 2
 Vec4: [156.1212701   79.76060688 111.76816996  68.8152313 ]
 Decay Rate: tensor(0.1226)
 Mass Squared: tensor(784.4371)

Node 3
 Vec4: [111.60604114  67.62854266  50.82814186  72.78235237]
 Decay Rate: tensor(0.1389)
 Mass Squared: tensor(1.5178)

Node 4
 Vec4: [140.19422275  83.55094254  68.34378123  89.34250807]
 Decay Rate: tensor(0.3355)
 Mass Squared: tensor(20.7039)

Node 5
 Vec4: [47.59900244 26.83523156 30.45806669 24.69359201]
 Decay Rate: tensor(0.1410)
 Mass Squared: tensor(8.0681)

Node 6
 Vec4: [108.5222783   52.92538075  81.31011088  44.12164398]
 Decay Rate: tensor(0.5328)
 Mass Squared: tensor(417.9355)

Node 7
 Vec4: [113.71670191  68.67541305  55.61891912  71.53689863]
 Decay Rate: tensor(0.3146)
 Mass Squared: tensor(4.183